In [1]:
from app_db_manager import (AppDbManager,
                            APP_STORE_CATEGORY_CASE_STATEMENT,
                            APP_STORE_CATEGORY_FILTER_STATEMENT,
                            GOOGLE_PLAY_CATEGORY_CASE_STATEMENT,
                            GOOGLE_PLAY_CATEGORY_FILTER_STATEMENT,
                           )

In [18]:
import pandas as pd

In [2]:
db = AppDbManager()

__init__ function with app_store_file=appleAppData.csv, google_play_file=Google-Playstore.csv


In [3]:
app_store_table_name = "app_store_apps_table"
google_play_table_name = "google_play_apps_table"

In [4]:
print(APP_STORE_CATEGORY_CASE_STATEMENT)
print(APP_STORE_CATEGORY_FILTER_STATEMENT)
print(GOOGLE_PLAY_CATEGORY_CASE_STATEMENT)
print(GOOGLE_PLAY_CATEGORY_FILTER_STATEMENT)


CASE
    WHEN category = 'Games' THEN 'Game'
    WHEN category = 'Music' THEN 'Music'
    WHEN category = 'Health & Fitness' THEN 'Health'
END


category IN ('Games', 'Music', 'Health & Fitness')


CASE
    WHEN category IN ('Games', 'Action', 'Adventure', 'Arcade',
                      'Board', 'Card', 'Casino, Casual',
                      'Educational', 'Music', 'Puzzle', 'Racing',
                      'Role Playing', 'Simulation', 'Sports',
                      'Strategy', 'Trivia', 'Word') THEN 'Game'
    WHEN category = 'Music & Audio' THEN 'Music'
    WHEN category = 'Health & Fitness' THEN 'Health'
END


category IN ('Games', 'Action', 'Adventure', 'Arcade',
                      'Board', 'Card', 'Casino, Casual',
                      'Educational', 'Music', 'Puzzle', 'Racing',
                      'Role Playing', 'Simulation', 'Sports',
                      'Strategy', 'Trivia', 'Word', 'Music & Audio', 'Health & Fitness')



## Plot the distribution of app sizes across our three categories.

In [4]:
query_1_1 = """
SELECT
    {case_statement} AS category,
    AVG(size_in_mb) AS avg_size,
    MIN(size_in_mb) AS min_size,
    MAX(size_in_mb) AS max_size
FROM {table_name}
WHERE {filter_statement}
GROUP BY 1
"""


In [5]:
# app store
result = db.run_select_pd(query_1_1.format(
    case_statement=APP_STORE_CATEGORY_CASE_STATEMENT,
    filter_statement=APP_STORE_CATEGORY_FILTER_STATEMENT,
    table_name=app_store_table_name,
))
print(result)

  category    avg_size  min_size      max_size
0     Game  142.563163  0.000000  73222.591724
1   Health   63.638878  0.143153   6351.762695
2    Music   73.180113  0.026348   5087.920526


In [6]:
# google play
result = db.run_select_pd(query_1_1.format(
    case_statement=GOOGLE_PLAY_CATEGORY_CASE_STATEMENT,
    filter_statement=GOOGLE_PLAY_CATEGORY_FILTER_STATEMENT,
    table_name=google_play_table_name,
))
print(result)

  category   avg_size  min_size  max_size
0     Game  30.556361  0.009766    1536.0
1   Health  28.802659  0.007520     706.0
2    Music  17.923657  0.008887    1126.4


## Compute the average rating across all ratings for each of the three categories.

In [7]:
query_2 = """
SELECT
    {case_statement} AS category,
    SUM(rating*rating_count) * 1.0 / SUM(rating_count) AS average_rating
FROM {table_name}
WHERE {filter_statement}
GROUP BY 1
"""

In [8]:
# app store
result = db.run_select_pd(query_2.format(
    case_statement=APP_STORE_CATEGORY_CASE_STATEMENT,
    filter_statement=APP_STORE_CATEGORY_FILTER_STATEMENT,
    table_name=app_store_table_name,
))
print(result)

  category  average_rating
0     Game        4.620479
1   Health        4.754431
2    Music        4.767138


In [9]:
# google play
result = db.run_select_pd(query_2.format(
    case_statement=GOOGLE_PLAY_CATEGORY_CASE_STATEMENT,
    filter_statement=GOOGLE_PLAY_CATEGORY_FILTER_STATEMENT,
    table_name=google_play_table_name,
))
print(result)

  category  average_rating
0     Game        4.272284
1   Health        4.474643
2    Music        4.387132


## Plot the number of new releases in each category over time by month

In [10]:
query_3 = """
SELECT
    {case_statement} AS category,
    SUBSTR(released_date, 1, 7) AS released_month,
    count(1) AS cnt
FROM {table_name}
WHERE {filter_statement}
GROUP BY 1, 2
ORDER BY 1, 2
"""

In [11]:
# app store
result = db.run_select_pd(query_3.format(
    case_statement=APP_STORE_CATEGORY_CASE_STATEMENT,
    filter_statement=APP_STORE_CATEGORY_FILTER_STATEMENT,
    table_name=app_store_table_name,
))
print(result)

    category released_month  cnt
0       Game                   3
1       Game        2008-09    5
2       Game        2008-10   42
3       Game        2008-11   41
4       Game        2008-12   42
..       ...            ...  ...
480    Music        2021-07  347
481    Music        2021-08  354
482    Music        2021-09  309
483    Music        2021-10  117
484    Music        2021-11    2

[485 rows x 3 columns]


In [12]:
# google play
result = db.run_select_pd(query_3.format(
    case_statement=GOOGLE_PLAY_CATEGORY_CASE_STATEMENT,
    filter_statement=GOOGLE_PLAY_CATEGORY_FILTER_STATEMENT,
    table_name=google_play_table_name,
))
print(result)

    category released_month   cnt
0       Game           None  9790
1       Game        2010-01     4
2       Game        2010-02    24
3       Game        2010-03    30
4       Game        2010-04    38
..       ...            ...   ...
410    Music        2021-02  2022
411    Music        2021-03  2060
412    Music        2021-04  1981
413    Music        2021-05  1500
414    Music        2021-06   380

[415 rows x 3 columns]


## List the 10 biggest apps by size in each category by year.

In [13]:
query_4 = """
WITH max_size_by_category_by_year AS (
    SELECT
        {case_statement} AS new_category,
        SUBSTR(released_date, 1, 4) AS released_year,
        MAX(size_in_mb) AS max_size
    FROM {table_name}
    WHERE {filter_statement}
    GROUP BY 1, 2
)
SELECT
    max_size_by_category_by_year.new_category AS category,
    SUBSTR(released_date, 1, 4) AS released_year,
    raw.app_name,
    raw.app_id,
    raw.size_in_mb
FROM {table_name} as raw
INNER JOIN max_size_by_category_by_year
    ON max_size_by_category_by_year.new_category = {case_statement}
    AND max_size_by_category_by_year.released_year = SUBSTR(raw.released_date, 1, 4)
    AND max_size_by_category_by_year.max_size <= raw.size_in_mb
ORDER BY 1, 2
"""

In [19]:
# app store
result = db.run_select_pd(query_4.format(
    case_statement=APP_STORE_CATEGORY_CASE_STATEMENT,
    filter_statement=APP_STORE_CATEGORY_FILTER_STATEMENT,
    table_name=app_store_table_name,
))
print(result)

   category released_year                               app_name   
0      Game          2008                                  QBeez  \
1      Game          2009                              Space Ace   
2      Game          2010                       Dragon's Lair HD   
3      Game          2011                     X3 Terran Conflict   
4      Game          2012                          Borderlands 2   
5      Game          2013                    Total War: NAPOLEON   
6      Game          2014                Divinity - Original Sin   
7      Game          2015         Company of Heroes 2 Collection   
8      Game          2016                                 XCOM 2   
9      Game          2017                              Mafia III   
10     Game          2018          Aerofly FS 2 Flight Simulator   
11     Game          2019              Total War: THREE KINGDOMS   
12     Game          2020                Total War: WARHAMMER II   
13     Game          2021                       

In [16]:
# google play
result = db.run_select_pd(query_4.format(
    case_statement=GOOGLE_PLAY_CATEGORY_CASE_STATEMENT,
    filter_statement=GOOGLE_PLAY_CATEGORY_FILTER_STATEMENT,
    table_name=google_play_table_name,
))
print(result)

   category released_year                                           app_name   
0      Game          2010    Yahoo Fantasy Sports: Football, Baseball & More  \
1      Game          2011                                                TSN   
2      Game          2012                                  Klaverjas HD Free   
3      Game          2013                                        城堡爭霸 - 聯盟霸業   
4      Game          2014                                      Summoners War   
5      Game          2015  Critical Ops: Online Multiplayer FPS Shooting ...   
6      Game          2016                                        Titan Quest   
7      Game          2017                   Guns of Boom - Online PvP Action   
8      Game          2018  Kiddopia: Preschool Education & ABC Games for ...   
9      Game          2019          Sword Art Online Alicization Rising Steel   
10     Game          2020                         Sky: Children of the Light   
11     Game          2021               